## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lavrentiya,RU,65.5833,-171.0000,31.44,78,19,6.06,few clouds
1,1,Kahului,US,20.8947,-156.4700,87.82,58,20,23.02,few clouds
2,2,Fulton,US,33.7668,-84.4499,71.26,61,0,5.75,clear sky
3,3,Rikitea,PF,-23.1203,-134.9692,76.69,70,41,11.03,scattered clouds
4,4,Bluff,NZ,-46.6000,168.3333,53.80,65,82,3.51,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Kahului,US,20.8947,-156.4700,87.82,58,20,23.02,few clouds
3,3,Rikitea,PF,-23.1203,-134.9692,76.69,70,41,11.03,scattered clouds
7,7,Pangai,TO,-19.8000,-174.3500,80.71,50,20,5.75,few clouds
10,10,Rae Bareli,IN,26.2167,81.2333,87.44,38,0,10.29,clear sky
11,11,Hilo,US,19.7297,-155.0900,82.27,60,20,10.00,few clouds
13,13,Luba,GQ,3.4568,8.5547,75.09,82,100,3.42,overcast clouds
20,20,Dzilam Gonzalez,MX,21.2833,-88.9333,83.61,65,16,10.33,few clouds
27,27,Lumphat,KH,13.4878,106.9816,76.87,91,100,2.42,overcast clouds
31,31,Dawei,MM,14.0833,98.2000,80.83,82,24,2.57,few clouds
38,38,Soyo,AO,-6.1349,12.3689,77.74,92,95,6.64,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                161
City                   161
Country                161
Lat                    161
Lng                    161
Max Temp               161
Humidity               161
Cloudiness             161
Wind Speed             161
Current Description    161
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()

In [33]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = "Hotel Name"
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kahului,US,87.82,few clouds,20.8947,-156.4700,Hotel Name
3,Rikitea,PF,76.69,scattered clouds,-23.1203,-134.9692,Hotel Name
7,Pangai,TO,80.71,few clouds,-19.8000,-174.3500,Hotel Name
10,Rae Bareli,IN,87.44,clear sky,26.2167,81.2333,Hotel Name
11,Hilo,US,82.27,few clouds,19.7297,-155.0900,Hotel Name
...,...,...,...,...,...,...,...
557,Mranggen,ID,84.04,few clouds,-7.0268,110.5158,Hotel Name
558,Mogadishu,SO,81.00,light rain,2.0371,45.3438,Hotel Name
560,Atambua,ID,81.39,scattered clouds,-9.1061,124.8925,Hotel Name
564,Kieta,PG,85.57,few clouds,-6.2167,155.6333,Hotel Name


In [36]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [37]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kahului,US,87.82,few clouds,20.8947,-156.4700,Maui Seaside Hotel
3,Rikitea,PF,76.69,scattered clouds,-23.1203,-134.9692,People ThankYou
7,Pangai,TO,80.71,few clouds,-19.8000,-174.3500,Ha'apai Beach Resort
10,Rae Bareli,IN,87.44,clear sky,26.2167,81.2333,Hotel Ashirwad Inn
11,Hilo,US,82.27,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
...,...,...,...,...,...,...,...
557,Mranggen,ID,84.04,few clouds,-7.0268,110.5158,Bushro Adventure - Toko Alat Outdoor Demak
558,Mogadishu,SO,81.00,light rain,2.0371,45.3438,Hotel Juba
560,Atambua,ID,81.39,scattered clouds,-9.1061,124.8925,Kost Alfa Flat
564,Kieta,PG,85.57,few clouds,-6.2167,155.6333,Uruna Bay Retreat


In [38]:
# 7. Drop the rows where there is no Hotel Name.
hotel_clean_df = hotel_df.dropna()
hotel_clean_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kahului,US,87.82,few clouds,20.8947,-156.4700,Maui Seaside Hotel
3,Rikitea,PF,76.69,scattered clouds,-23.1203,-134.9692,People ThankYou
7,Pangai,TO,80.71,few clouds,-19.8000,-174.3500,Ha'apai Beach Resort
10,Rae Bareli,IN,87.44,clear sky,26.2167,81.2333,Hotel Ashirwad Inn
11,Hilo,US,82.27,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
...,...,...,...,...,...,...,...
557,Mranggen,ID,84.04,few clouds,-7.0268,110.5158,Bushro Adventure - Toko Alat Outdoor Demak
558,Mogadishu,SO,81.00,light rain,2.0371,45.3438,Hotel Juba
560,Atambua,ID,81.39,scattered clouds,-9.1061,124.8925,Kost Alfa Flat
564,Kieta,PG,85.57,few clouds,-6.2167,155.6333,Uruna Bay Retreat


In [40]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_clean_df.to_csv(output_data_file, index_label="City_ID")

In [43]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_clean_df[["Lat", "Lng"]]

In [45]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))